In [88]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModel
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
# 加载多任务数据集（7分类）
multi_task_df = pd.read_csv("cti.csv")
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import torch.nn.functional as F

In [70]:
multi_task_df

,smiles,Primary qHTS for inhibitors of nuclear receptor binding SET domain protein 2 (NSD2) in an isogenic RCH-ACV NSD2 p.E1099K mutant (9B),"Caspase-3/7 induction in CHO-K1 cells by small molecules, qHTS cell viability counter screen",Cytochrome P450 Family 2 Subfamily C Member 9 (CYP2C9) small molecule antagonists: luciferase reporter qHTS assay,Cytochrome P450 Family 3 Subfamily A Member 4 (CYP3A4) small molecule antagonists: luciferase reporter qHTS assay,Cytochrome P450 family 2 subfamily C member 9 (CYP2C9) small molecule antagonists: luciferase cell-based qHTS assay,SW480 viability from Cell TiterGlo-screen,qHTS assay to identify small molecule agonists of the estrogen receptor beta (ER-beta) signaling pathway - cell viability counter screen
0,CC1OC(OC2C(O)C(O)C(OC2CO)OC3C(O)C(O)C(O)OC3CO)...,0,0,0,0,0,0,0
1,[H][n]1c(NC(=O)OC)nc2ccc(SCCC)cc21,1,1,1,1,1,1,1
2,CC(N)Cc1ccccc1,1,1,1,1,1,1,1
3,CNCCC(Oc1ccccc1C)c2ccccc2,1,1,1,1,1,0,1
4,NCC(CC(O)=O)c1ccc(Cl)cc1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...
657,CC1OC1P(O)(O)=O,0,0,1,0,1,0,1
658,OP(O)(=O)OCN1C(=O)NC(C1=O)(c2ccccc2)c3ccccc3,1,0,0,0,0,0,0
659,NC1C=CN(C2OC(CO)C(O)C2(F)F)C(=O)N=1,0,1,0,1,0,1,0
660,CCN(CCO)CCCC(C)Nc1ccnc2cc(Cl)ccc12,0,0,1,1,1,0,1


In [72]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# 指定本地模型路径
model_path = "D:\jupyter-project\Transfer-Learning-ChemBERTa-main\DeepChem\ChemBERTa-77M-MTR"

# 从本地路径加载分词器
tokenizer = AutoTokenizer.from_pretrained(model_path)
# 从本地路径加载模型配置并设置标签数量和输出隐藏状态
config = AutoConfig.from_pretrained(model_path)
config.num_labels = 7
config.output_hidden_states = True

# 从本地路径加载具有更新配置的模型
model = AutoModelForSequenceClassification.from_pretrained(model_path, config=config)

print("分词器和模型加载成功。")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at D:\jupyter-project\Transfer-Learning-ChemBERTa-main\DeepChem\ChemBERTa-77M-MTR and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


分词器和模型加载成功。


In [73]:
# 标记化输入数据
inputs = tokenizer(list(multi_task_df["smiles"]), padding=True, truncation=True, return_tensors="pt")
labels = torch.tensor(multi_task_df.iloc[:, 1:].values, dtype=torch.float)

In [74]:
labels

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [0., 1., 0.,  ..., 0., 1., 0.],
        [0., 0., 1.,  ..., 1., 0., 1.],
        [1., 0., 0.,  ..., 1., 0., 0.]])

In [75]:
inputs

{'input_ids': tensor([[12, 16, 16,  ...,  0,  0,  0],
        [12, 25, 20,  ...,  0,  0,  0],
        [12, 16, 16,  ...,  0,  0,  0],
        ...,
        [12, 23, 16,  ...,  0,  0,  0],
        [12, 16, 16,  ...,  0,  0,  0],
        [12, 19, 16,  ...,  0,  0,  0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [76]:
# 创建数据集对象
class MultiTaskDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.inputs.items()}
        item['labels'] = self.labels[idx]
        return item

dataset = MultiTaskDataset(inputs, labels)

In [77]:
# 创建数据加载器
batch_size = 8
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# 定义优化器
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

# 训练模型
model.train()
num_epochs = 3
for epoch in range(num_epochs):
    for batch in dataloader:
        optimizer.zero_grad()
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}")

# 获取化学分子的表示和预测值
model.eval()
with torch.no_grad():
    outputs = model(**inputs)
    embeddings = outputs.hidden_states[-1][:, 0, :]  # 获取 [CLS] token 的表示
    predictions = torch.sigmoid(outputs.logits)  # 获取每个标签的预测概率

# 拼接表示和预测值
enhanced_embeddings = torch.cat((embeddings, predictions), dim=1)

Epoch 1/3, Loss: 0.629591703414917
Epoch 2/3, Loss: 0.7051559090614319
Epoch 3/3, Loss: 0.6541862487792969


In [57]:
a=pd.DataFrame(enhanced_embeddings)

In [58]:
a

,0,1,2,3,4,5,6,7,8,9,...,381,382,383,384,385,386,387,388,389,390
0,-0.346450,1.123505,0.070002,0.360900,0.605828,-0.675381,0.514018,0.512861,0.131690,-1.039229,...,-0.743365,0.321951,-0.047417,0.217071,0.258271,0.219865,0.251743,0.245754,0.279263,0.221365
1,-0.091667,0.432119,-0.087942,0.304289,0.298097,-0.035807,0.156919,0.438907,0.284944,-0.286521,...,0.478680,-0.309472,0.297819,0.382892,0.386959,0.364506,0.340307,0.335903,0.377132,0.359698
2,-0.020884,0.547709,0.047136,0.410430,-0.159501,-0.100027,0.038059,0.306977,0.276461,-0.662162,...,0.099500,-0.189193,0.269075,0.368493,0.379968,0.355579,0.367586,0.346683,0.381523,0.345651
3,0.088647,0.407110,0.028807,0.628901,0.073800,0.085578,-0.379275,0.410329,0.081060,-0.306328,...,-0.039222,0.033633,0.121538,0.344041,0.371199,0.359230,0.367141,0.346833,0.380919,0.355888
4,-0.235635,0.488055,-0.071355,0.243413,0.099301,-0.207267,0.028496,0.386522,0.164248,-0.315484,...,-0.215535,-0.378307,0.231053,0.320040,0.347490,0.328069,0.335393,0.326314,0.351804,0.324369
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
657,-0.317899,0.373583,-0.000684,0.653361,0.200163,0.089872,0.374648,0.348040,0.118031,-0.689553,...,-0.435049,-0.468634,-0.199697,0.368845,0.366350,0.347385,0.326457,0.336934,0.381191,0.347766
658,0.071228,0.590029,-0.184651,0.762904,-0.409814,0.062142,0.177005,1.263602,-0.013780,0.157264,...,-0.010946,-0.045272,0.687275,0.327855,0.331398,0.332792,0.281315,0.272426,0.347421,0.311751
659,0.209132,0.455391,-0.154186,0.446514,0.211825,-0.285499,0.412272,0.223319,0.263184,-0.441083,...,-0.207180,0.117185,0.496743,0.324579,0.329340,0.326665,0.310185,0.313503,0.355455,0.305777
660,0.000436,0.021608,0.029461,0.383327,0.334455,-0.353142,-0.132224,0.273075,0.056448,-0.204536,...,0.055722,-0.142515,0.240393,0.357940,0.396429,0.361156,0.390793,0.375501,0.394118,0.362248


In [84]:
# 加载二分类数据集
binary_df = pd.read_csv("data2.csv")
from sklearn.model_selection import train_test_split
# 划分数据集为训练集和测试集
train_binary_df, test_binary_df = train_test_split(binary_df, test_size=0.2, random_state=42)
# 标记化输入数据
binary_inputs = tokenizer(list(binary_df["smiles"]), padding=True, truncation=True, return_tensors="pt")

# 标记化训练集和测试集输入数据
train_binary_inputs = tokenizer(list(train_binary_df["smiles"]), padding=True, truncation=True, return_tensors="pt")
test_binary_inputs = tokenizer(list(test_binary_df["smiles"]), padding=True, truncation=True, return_tensors="pt")

# 获取训练集和测试集的增强表示
model.eval()
with torch.no_grad():
    train_binary_outputs = model(**train_binary_inputs)
    train_binary_embeddings = train_binary_outputs.hidden_states[-1][:, 0, :]  # 获取 [CLS] token 的表示
    train_binary_predictions = torch.sigmoid(train_binary_outputs.logits)  # 获取每个标签的预测概率
    train_binary_predictions = (train_binary_predictions > 0.5).float()
    train_binary_enhanced_embeddings = torch.cat((train_binary_embeddings, train_binary_predictions), dim=1)
    
    test_binary_outputs = model(**test_binary_inputs)
    test_binary_embeddings = test_binary_outputs.hidden_states[-1][:, 0, :]  # 获取 [CLS] token 的表示
    test_binary_predictions = torch.sigmoid(test_binary_outputs.logits)  # 获取每个标签的预测概率
    test_binary_predictions = (test_binary_predictions > 0.5).float()
    test_binary_enhanced_embeddings = torch.cat((test_binary_embeddings, test_binary_predictions), dim=1)

print("Train binary enhanced embeddings shape:", train_binary_enhanced_embeddings.shape)
print("Test binary enhanced embeddings shape:", test_binary_enhanced_embeddings.shape)


Train binary enhanced embeddings shape: torch.Size([502, 391])
Test binary enhanced embeddings shape: torch.Size([126, 391])


In [85]:
# 创建新的数据集对象
train_binary_labels = torch.tensor(train_binary_df["label"].values, dtype=torch.float)
test_binary_labels = torch.tensor(test_binary_df["label"].values, dtype=torch.float)

class BinaryDataset(Dataset):
    def __init__(self, embeddings, labels):
        self.embeddings = embeddings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {'input_ids': self.embeddings[idx], 'labels': self.labels[idx]}

train_binary_dataset = BinaryDataset(train_binary_enhanced_embeddings, train_binary_labels)
test_binary_dataset = BinaryDataset(test_binary_enhanced_embeddings, test_binary_labels)

In [86]:
from torch import nn, optim

class BinaryClassificationModel(nn.Module):
    def __init__(self, input_dim):
        super(BinaryClassificationModel, self).__init__()
        self.linear = nn.Linear(input_dim, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.linear(x)
        x = self.sigmoid(x)
        return x

binary_model = BinaryClassificationModel(train_binary_enhanced_embeddings.shape[1])

# 定义训练参数和优化器
optimizer = optim.Adam(binary_model.parameters(), lr=1e-3)
criterion = nn.BCELoss()

# 训练模型
def train_binary_model(model, dataset, optimizer, criterion, epochs=3):
    model.train()
    dataloader = DataLoader(dataset, batch_size=8, shuffle=True)
    for epoch in range(epochs):
        total_loss = 0
        for batch in dataloader:
            optimizer.zero_grad()
            inputs, labels = batch['input_ids'], batch['labels']
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f'Epoch {epoch + 1}, Loss: {total_loss / len(dataloader)}')

train_binary_model(binary_model, train_binary_dataset, optimizer, criterion, epochs=3)

Epoch 1, Loss: 0.2972847366380313
Epoch 2, Loss: 0.16776035918248078
Epoch 3, Loss: 0.1598043256099262


In [87]:
# 进行预测
binary_model.eval()
with torch.no_grad():
    test_predictions = binary_model(test_binary_enhanced_embeddings).squeeze()

# 将预测值转为0和1
test_predictions = (test_predictions > 0.5).float()
print(test_predictions)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.])
